In [7]:
from IPython.core.display import display, HTML
display(HTML('<h1>Image Feature Extraction</h1>'))



In [8]:
display(HTML('<h2>Imports</h2>'))

In [10]:
import glob
import pandas as pd
import numpy as np
import cv2
from skimage import feature

In [11]:
display(HTML('<h2>Local Binary Pattern</h2>'))

In [12]:
def LBP1(image):
    lbp = feature.local_binary_pattern(image, 24,8,method="uniform")
    (hist, _) = np.histogram(lbp.ravel(),bins=np.arange(0, 24 + 3),range=(0, 24 + 2))
    return hist

In [13]:
display(HTML('<h2>Histogram Of Oriented Gradients</h2>'))

In [14]:
def HOG(image):
    winSize = (64,64)
    blockSize = (16,16)
    blockStride = (8,8)
    cellSize = (8,8)
    nbins = 9
    derivAperture = 1
    winSigma = 4.
    histogramNormType = 0
    L2HysThreshold = 2.0000000000000001e-01
    gammaCorrection = 0
    nlevels = 64
    hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,derivAperture,winSigma,
                            histogramNormType,L2HysThreshold,gammaCorrection,nlevels)
    winStride = (8,8)
    padding = (8,8)
    locations = ((10,20),)
    hist = hog.compute(image,winStride,padding,locations)
    
    return hist

In [15]:
display(HTML('<h2>Histogram</h2>'))

In [16]:
def histogram(gray_img):
    x=np.histogram(gray_img, bins = 10)[0]
    x2=(np.histogram(gray_img, bins = 10)[1])
    
    return np.concatenate([x,x2])


In [18]:
display(HTML('<h2>Combine All IP Techinques</h2>'))

In [17]:
def make_data(raw_pixels_df):
    new_pixels = pd.DataFrame()
    new_pixels['ALL'] = raw_pixels_df['raw_pixels'].apply(lambda x : np.concatenate([LBP1(x).reshape(1,26),HOG(x).reshape(1,1764),histogram(x).reshape(1,21)], axis = 1))
    new_pixels['name'] = raw_pixels_df['name']
    return new_pixels

In [6]:
def csv_maker(raw_pixels_df):
    final = pd.DataFrame()
    raw_pixels_df = make_data(raw_pixels_df)
    final['value'] = raw_pixels_df['ALL']
    final['name'] = raw_pixels_df[['name']]
    final['value'] = final['value'].apply(lambda x : x[0])
    csv_format = pd.DataFrame()
    csv_format[["x"+str(i) for i in range(1811)]] = pd.DataFrame(final.value.values.tolist(), index = final.index)
    csv_format['name'] = final['name']
    return csv_format
   

In [19]:
display(HTML('<h2>Main Thread</h2>'))

In [22]:
filenames = glob.glob('/home/sagar/Desktop/BE/BE Project/images/*.jpg')

counter = 0
for i in range(len(filenames)//100):
    raw_pixels = []
    names = []
    for j in range(100):    
        raw_pixels.append(cv2.imread(filenames[counter],cv2.IMREAD_GRAYSCALE))
        names.append(filenames[counter].split("/")[-1])
        counter = counter + 1
    raw_pixels_df = pd.DataFrame()
    raw_pixels_df['raw_pixels'] = raw_pixels
    raw_pixels_df['name']= names
    print((counter/5)*100)
    x=(csv_maker(raw_pixels_df))
    if counter == 100:
        x.to_csv('my_csv.csv')
    else:
        x.to_csv('my_csv.csv', mode='a', header=False)

    


